In [1]:
from pathlib import Path
import geopandas as gpd
import shapely
import folium
import seaborn as sns

So basically i got the request to do the inference of Sentinel 2 in the area which is covered by "tiles_nwt_2010_2016.geojson":

In [6]:
gpd_request = (gpd.GeoDataFrame.from_file("tiles_nwt_2010_2016.geojson")
               .dissolve().explode(ignore_index=True,index_parts=True)
               ).drop(columns="tile_name")
gpd_request["name"] = gpd_request.index.map(lambda x: f"{x:02d}")
display(gpd_request)
gpd_request.explore()

,geometry,name
0,"POLYGON ((-2540000 10000, -2550000 10000, -256...",00
1,"POLYGON ((-2340000 -80000, -2350000 -80000, -2...",01


In [4]:
s2_coverage = gpd.read_parquet("Sentinel2_Tiles_Arctic.parquet")
s2_coverage

,Name,geometry
957,01VCH,"MULTIPOLYGON (((180 60.3116, 179.38037 60.2993..."
958,01VCJ,"MULTIPOLYGON (((180 61.20937, 179.27794 61.194..."
959,01VCK,"MULTIPOLYGON (((180 62.10763, 179.16853 62.090..."
960,01VCL,"MULTIPOLYGON (((180 63.00584, 179.0515 62.9866..."
966,01VDH,"MULTIPOLYGON (((-178.86875 61.32159, -176.8176..."
...,...,...
56681,60XWM,"MULTIPOLYGON (((180 77.36401, 176.99918 77.389..."
56682,60XWN,"MULTIPOLYGON (((180 78.26016, 176.99912 78.285..."
56683,60XWP,"MULTIPOLYGON (((180 79.15627, 176.99905 79.181..."
56684,60XWQ,"MULTIPOLYGON (((180 80.05179, 176.99896 80.077..."


Selecting the Sentinel 2 TILE ids which cover our area of interest.

In [7]:

s2_tiles = s2_coverage.sjoin(gpd_request.to_crs(4326)).drop_duplicates("Name")

minx, miny, maxx, maxy = s2_tiles.total_bounds
m = folium.Map()

s2_tiles.assign(utm=s2_tiles.Name.str[0:2]).explore(m=m, column="utm", attr="Name")
gpd_request.explore(m=m, color="red")
m.fit_bounds([[miny,minx],[maxy,maxx]])
m

i drop the two tiles which are in utm zone 7 (blue tiles), and select the six tiles which make up the coverage (08WMA is not necessary):

In [8]:
s2_ids = [s2name for s2name in s2_tiles.Name if s2name.startswith("08")] # only UTM08
s2_ids = list(set(s2_ids) - {"08WMA"}) # 08WMA is not necessary
s2_ids

['08WMU', '08WNC', '08WMV', '08WNV', '08WNA', '08WNB']

In [10]:
import ee
ee.Initialize()

Querying for:
* tile id ("MGRS_TILE") in s2_ids 
* June - September
* cloudy pixel percentage below 10

map_ic_to_fc() just converts me the Image info into a FeatureCollection/GeoPandas GeoDataFrame

In [12]:
def map_ic_to_fc(img:ee.Image):
    return ee.Feature(img.geometry(),{
        "ee_id" : img.id(),
        "ee_timestamp" : img.get("system:time_start"),
        "tile_id" : img.get("MGRS_TILE"),
        "ee_cpp" : img.get("CLOUDY_PIXEL_PERCENTAGE"),
        "ee_datatake": img.get("DATATAKE_IDENTIFIER"),
        "ee_quality" : img.get("GENERAL_QUALITY")
    })

ee_ic = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filter(ee.Filter.And(
    ee.Filter.inList("MGRS_TILE", s2_ids ),
    ee.Filter.calendarRange(6,9,"month"),
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)
))

ee_fc = ee.FeatureCollection(  ee_ic.map(map_ic_to_fc) )

gdf_ee_s2ftrs = ee.data.computeFeatures({
    "expression" : ee_fc,
    "fileFormat" : "GEOPANDAS_GEODATAFRAME"
}).set_crs(4326)

gdf_ee_s2ftrs

,geometry,ee_cpp,ee_datatake,ee_id,ee_quality,ee_timestamp,tile_id
0,"POLYGON ((-137.35303 67.59851, -137.31964 67.2...",0.432087,GS2A_20190601T211521_020589_N02.12,20190601T211521_20190601T211516_T08WMV,PASSED,1559423777000,08WMV
1,"POLYGON ((-135.00077 69.40966, -135.00074 68.5...",0.230567,GS2A_20190601T211521_020589_N02.12,20190601T211521_20190601T211516_T08WNB,PASSED,1559423745000,08WNB
2,"POLYGON ((-134.04709 67.52518, -134.04699 67.5...",2.819972,GS2B_20190604T202849_011723_N02.12,20190604T202849_20190604T202903_T08WNA,PASSED,1559680575000,08WNA
3,"POLYGON ((-133.13132 68.41466, -133.13113 68.4...",8.008250,GS2B_20190604T202849_011723_N02.12,20190604T202849_20190604T202903_T08WNB,PASSED,1559680563000,08WNB
4,"POLYGON ((-135.0008 70.30602, -135.00076 69.32...",4.962671,GS2A_20190605T205021_020646_N02.12,20190605T205021_20190605T205023_T08WNC,PASSED,1559768138000,08WNC
...,...,...,...,...,...,...,...
530,"POLYGON ((-137.35303 67.59851, -137.32883 67.3...",8.838084,GS2B_20240904T210029_039165_N05.11,20240904T210029_20240904T210025_T08WMV,PASSED,1725483982962,08WMV
531,"POLYGON ((-135.00073 68.51265, -135.0007 67.52...",7.288944,GS2B_20240904T210029_039165_N05.11,20240904T210029_20240904T210025_T08WNA,PASSED,1725483962929,08WNA
532,"POLYGON ((-135.00077 69.40964, -135.00073 68.4...",7.134860,GS2B_20240911T204939_039265_N05.11,20240911T204939_20240911T204937_T08WNB,PASSED,1726088151059,08WNB
533,"POLYGON ((-134.90307 66.6327, -134.90172 66.63...",8.887471,GS2B_20240915T203009_039322_N05.11,20240915T203009_20240915T203235_T08WNV,PASSED,1726432580057,08WNV


So these are the S2_ids we actually want to download:

In [13]:
gdf_ee_s2ftrs.ee_id

0      20190601T211521_20190601T211516_T08WMV
1      20190601T211521_20190601T211516_T08WNB
2      20190604T202849_20190604T202903_T08WNA
3      20190604T202849_20190604T202903_T08WNB
4      20190605T205021_20190605T205023_T08WNC
                        ...                  
530    20240904T210029_20240904T210025_T08WMV
531    20240904T210029_20240904T210025_T08WNA
532    20240911T204939_20240911T204937_T08WNB
533    20240915T203009_20240915T203235_T08WNV
534    20240920T212519_20240920T212515_T08WNC
Name: ee_id, Length: 535, dtype: object

So for download all these you could basically do this (not really tested but you get the gist)

In [ ]:
import geedim as gd

def download_with_basepath(s2_id, basepath):
    download_folder = Path(basepath) / s2_id
    download_folder.mkdir( parents=True,exist_ok=True )

    download_filepath = download_folder / f"{s2_id}_SR.tif"
    sr_ee_img = ee.Image("COPERNICUS/S2_SR_HARMONIZED/"+s2_id).select(["B2","B3","B4","B8"])
    gd_sr_img = gd.download.BaseImage(sr_ee_img)
    gd_sr_img.download( download_filepath, dtype='uint16', overwrite=True)

    scl_download_filepath = download_folder / f"{s2_id}_SCL.tif"
    scl_ee_img = ee.Image("COPERNICUS/S2_SR_HARMONIZED/"+s2_id).select(["SCL"])
    gd_scl_img = gd.download.BaseImage(scl_ee_img)
    gd_scl_img.download( scl_download_filepath, dtype='uint16', overwrite=True)

gdf_ee_s2ftrs.ee_id.apply(download_with_basepath, basepath = Path(r"C:\Temp"))
